## Installing and importing face_recognition

In [110]:
#!pip install dlib
#!pip install face_recognition
#!pip install imutils

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25853 sha256=1b513c90c20ec77d6f6a19711a7403153a24f4545f73405a5d3dd98ac8f14a4d
  Stored in directory: c:\users\sakib ahmed\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils


In [162]:
import dlib
import face_recognition
import cv2
import numpy as np
from PIL import Image
import os
import imutils
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow

## Encoding faces

In [134]:
photo_01 = face_recognition.load_image_file(r'.\data\cropped_images\Tanvir Mahtab.jpg')
photo_01_encoding = face_recognition.face_encodings(photo_01)[0]

In [135]:
len(photo_01_encoding)

128

In [136]:
unknown_photo = face_recognition.load_image_file(r'.\data\cropped_images\test\test 2.jpg')
unknown_encoding = face_recognition.face_encodings(unknown_photo)[0]

In [137]:
results = face_recognition.compare_faces([photo_01_encoding], unknown_encoding)
print(results)

[False]


In [138]:
photo_02 = face_recognition.load_image_file(r'.\data\cropped_images\Ahmad Nakib.jpg')
photo_02_encoding = face_recognition.face_encodings(photo_02)[0]

In [139]:
list_encodings = [photo_01_encoding, photo_02_encoding]

In [140]:
list_names = ['Tanvir Mahtab', 'Ahmad Nakib']

In [141]:
test_image = cv2.imread(r'.\data\cropped_images\test\test 1.jpg')
image_rgb = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
face_locations = face_recognition.face_locations(image_rgb, model = 'hog')
face_encoding = face_recognition.face_encodings(image_rgb, face_locations)
face_encoding = face_encoding[0]

In [142]:
results = face_recognition.compare_faces(list_encodings, face_encoding)
print(results)

[True, True]


The above codes are showing false positives. So I have implemented the following solution (np.argmin for minimum face distances):

In [145]:
face_distances = face_recognition.face_distance(list_encodings, face_encoding)
print(face_distances)

[0.40505838 0.59486724]


In [146]:
best_match_index = np.argmin(face_distances)
best_match_index

0

In [170]:
if results[best_match_index]:
  pred = list_names[best_match_index]
  print(pred + ' is in this picture')
  cv2.imshow('Test Image', test_image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  print(face_distances[best_match_index])

Ahmad Nakib is in this picture
0.4050583821332193


## Encoding all image's faces from a specific path

In [149]:
paths = [os.path.join('./data/cropped_images', f) for f in os.listdir('./data/cropped_images') if f.endswith('.jpg')]
print(paths)

['./data/cropped_images\\Ahmad Nakib.jpg', './data/cropped_images\\Tanvir Mahtab.jpg']


In [150]:
def get_encodings(paths):
  print('{} images found'.format(len(paths)))
  list_encodings = []
  list_names = []
  for img_path in paths:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    basename = os.path.basename(img_path)
    #print(basename)
    (name, ext) = os.path.splitext(basename)
    #print(name, ext)
    face_roi = face_recognition.face_locations(img, model = 'cnn') # hog
    face_encoding = face_recognition.face_encodings(img, face_roi)[0]
    if len(face_encoding) > 0:
      list_encodings.append(face_encoding)
      list_names.append(name)
    else:
      print('Could not detect the face from image {}'.format(img_path))
  return list_encodings, list_names

In [151]:
list_encodings, list_names = get_encodings(paths)

2 images found


In [152]:
len(list_encodings), len(list_names)

(2, 2)

In [153]:
list_names

['Ahmad Nakib', 'Tanvir Mahtab']

In [154]:
#list_encodings

In [155]:
def recognize_faces(image, list_encodings, list_names, tolerance = 0.6):
  img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  face_locations = face_recognition.face_locations(img_rgb)
  face_encodings = face_recognition.face_encodings(img_rgb, face_locations)
  face_names = []
  conf_values = []
  for encoding in face_encodings:
    matches = face_recognition.compare_faces(list_encodings, encoding, tolerance = tolerance)
    name = 'Not identified'
    face_distances = face_recognition.face_distance(list_encodings, encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
      name = list_names[best_match_index]
    face_names.append(name)
    conf_values.append(face_distances[best_match_index])
  face_locations = np.array(face_locations)
  return face_locations.astype(int), face_names, conf_values

## Display recognition above bounding box

- Coordinates are returned in this order: top, right, bottom, left ([ref](https://face-recognition.readthedocs.io/en/latest/face_recognition.html#face_recognition.api.face_locations))

In [156]:
test_image = cv2.imread(r'.\data\cropped_images\test\test 1.jpg')

In [157]:
face_locations, face_names, conf_values = recognize_faces(test_image, list_encodings, list_names)

In [158]:
#face_locations

In [159]:
face_names

['Tanvir Mahtab']

In [160]:
conf_values

[0.4315571251918219]

In [167]:
for face_loc, name, conf in zip(face_locations, face_names, conf_values):
  y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
  cv2.putText(test_image, name, (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0,0,255), 2)
  cv2.rectangle(test_image, (x1, y1), (x2, y2), (0,10,255), 4)
  print(conf)
cv2.imshow('Test_Image', test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.4315571251918219


Putting everything under a single function:

In [168]:
def show_recognition(test_image, list_encodings, list_names, max_width=700, tolerance=0.6):
  face_locations, face_names, conf_values = recognize_faces(test_image, list_encodings, list_names, tolerance)

  for face_loc, name, conf in zip(face_locations, face_names, conf_values):
    y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

    cv2.putText(test_image, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 0, 255), 2)
    cv2.rectangle(test_image, (x1, y1), (x2, y2), (0, 10, 255), 4)
    print(conf)

  if (test_image.shape[1] > max_width):
    test_image = imutils.resize(test_image, width=max_width)
  cv2.imshow('Test_Image',test_image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [171]:
test_image = cv2.imread(r'.\data\cropped_images\test\test 2.jpg')
show_recognition(test_image, list_encodings, list_names)

0.5194259466244441


### Recognize all faces from path



In [172]:
path_test = r".\data\cropped_images\test"
images_test = [os.path.join(path_test, f) for f in os.listdir(path_test)]
images_test

['.\\data\\cropped_images\\test\\test 1.jpg',
 '.\\data\\cropped_images\\test\\test 2.jpg',
 '.\\data\\cropped_images\\test\\test 3.jpg']

In [173]:
for image_path in images_test:
  print(image_path)
  test_image = cv2.imread(image_path)
  show_recognition(test_image, list_encodings, list_names)

.\data\cropped_images\test\test 1.jpg
0.4315571251918219
.\data\cropped_images\test\test 2.jpg
0.5194259466244441
.\data\cropped_images\test\test 3.jpg
0.4084710124456488


### Storing and loading encodings

In [122]:
import pickle
pickle_name = "user_list.pickle"
encodings_data = {"encodings": list_encodings, "names": list_names}
f = open(pickle_name, "wb")
f.write(pickle.dumps(encodings_data))
f.close()

In [123]:
data_encoding = pickle.loads(open(pickle_name, "rb").read())

In [124]:
list_encodings = data_encoding["encodings"]
list_names = data_encoding["names"]

In [125]:
len(list_encodings)

2

### Test recognition on image

In [174]:
image_path = r'.\data\cropped_images\test\test 2.jpg'
test_image = cv2.imread(image_path)
show_recognition(test_image, list_encodings, list_names)

0.5194259466244441
